In [1]:
from os import chdir
from pandas import DataFrame,read_csv,Series
chdir("/mnt/research/germs/shane/antibioticResistance/data")
# Load metadata
metadata = read_csv("tables/Metadata.csv")
metadata = metadata[metadata["Assay Type"].isin(set(['RNA-Seq', 'WGS']))]
metadata.set_index("Run",inplace=True)
metadata.drop(["AvgSpotLen","BioSampleModel","BioSample","BioSampleModel", \
               "Center Name","Consent", "DATASTORE filetype","DATASTORE provider","DATASTORE region",\
               "geo_loc_name","host_body_product","chem_administration","env_feature",'sub_species','host_life_stage',\
              "host_last_meal","samp_mat_process","material","host_growth_cond","project_name","investigation_type","label","Instrument",\
              "host_taxid",'host_body_temp', 'lat_lon', 'LibraryLayout','LibrarySelection', 'LibrarySource','Platform','ReleaseDate',\
              'samp_store_temp','BIOME','env_package','feature','host_common_name','env_biome', 'env_material', 'host','SRA Study',],inplace=True,axis=1)
treated = metadata[metadata["perturbation"]=="medicated"]
untreated = metadata[metadata["perturbation"]=="non-medicated"]

metadata.head()

,Assay Type,BioProject,Experiment,Host_Age,Host_Diet,host_family_relationship,host_sex,Library Name,MBases,MBytes,Organism,perturbation,sample_acc,Sample Name,collectiondate,samp_size,samp_store_dur,specific_host,collection_date
Run,,,,,,,,,,,,,,,,,,,
SRR5368774,RNA-Seq,PRJNA237795,SRX2664030,47 days,"TechStart 17-25\, Kent Feeds\, Muscatine\, IA",Offspring of sow B,male,200_7_feces_mt,5649,4041,pig gut metagenome,non-medicated,SRS2065942,200_7_feces,NaN,NaN,NaN,NaN,11-Jun-12
SRR5368775,WGS,PRJNA237795,SRX2664031,47 days,"TechStart 17-25\, Kent Feeds\, Muscatine\, IA",Offspring of sow B,male,200_7_feces_mg,80,56,pig gut metagenome,non-medicated,SRS2065942,200_7_feces,NaN,NaN,NaN,NaN,11-Jun-12
SRR5368778,RNA-Seq,PRJNA237795,SRX2664034,42 days,"TechStart 17-25\, Kent Feeds\, Muscatine\, IA",Offspring of sow B,male,200_2_feces_mt,5312,3610,pig gut metagenome,non-medicated,SRS2065944,200_2_feces,NaN,NaN,NaN,NaN,6-Jun-12
SRR5368779,WGS,PRJNA237795,SRX2664035,42 days,"TechStart 17-25\, Kent Feeds\, Muscatine\, IA",Offspring of sow B,male,200_2_feces_mg,90,62,pig gut metagenome,non-medicated,SRS2065944,200_2_feces,NaN,NaN,NaN,NaN,6-Jun-12
SRR5368780,RNA-Seq,PRJNA237795,SRX2664036,41 days,"TechStart 17-25\, Kent Feeds\, Muscatine\, IA",Offspring of sow B,male,200_1_feces_mt,7182,5152,pig gut metagenome,non-medicated,SRS2065945,200_1_feces,NaN,NaN,NaN,NaN,5-Jun-12


In [2]:
RNASeqSamples = metadata[metadata["Assay Type"] == "RNA-Seq"]
RNASeqSamples.shape

(47, 19)

In [123]:
class KrakenParser(DataFrame):
    def __init__(self,columns):
        super().__init__(columns=columns)
        self.set_index(["SampleName","Scientific Name"],inplace=True)
        
    def parse(self, sampleID, krakenOutput): 
        for rec in self.__parse_input(krakenOutput): 
            rID = (sampleID,rec[0])
            self.loc[rID,:]=rec[1:]

    def __parse_input(self,krakenOutput):
        with open(krakenOutput,'r') as fh:
            for line in fh:
                rec = line.strip().split('\t')
                percReads, cladeReads, assignedReads, rank, taxID, scientificName = rec
                percReads, cladeReads, assignedReads, scientificName = float(percReads), int(cladeReads), int(assignedReads), scientificName.replace(" ","")
                if assignedReads == 0: continue
#                 else:print(percReads, cladeReads, assignedReads, rank, taxID, scientificName,sep="\t|")
#                 yield {"%Reads":percReads, "Clade Reads":cladeReads, "ReadCount":assignedReads, "Rank":rank, "TaxID":taxID, "Scientific Name":scientificName, "Internal Node?":taxID==scientificName}
                yield [scientificName, percReads, cladeReads, assignedReads, rank, taxID, taxID==scientificName]               
                

In [125]:
parser = KrakenParser(columns=["SampleName","%Reads","Clade Reads","ReadCount","Rank","TaxID","Scientific Name","Internal Node?"])
for sampID in RNASeqSamples.index:
    treated = RNASeqSamples.at[sampID,"perturbation"]=="medicated"
    outFolder = "untreatedCounts"
    if treated: outFolder = "treatedCounts"
    for prot in ["cas12"]:
        parser.parse(sampID, "tables/%s/%s.%s.report" % (outFolder,sampID,prot))

parser.head()

%Reads Clade Reads ReadCount Rank  \
SampleName Scientific Name                                                  
SRR5368774 NZ_CABJFM010000001.1_ORF1622  18.75          51        51    -   
           757                            0.37           1         1    -   
           NZ_VZBX01000029.1_ORF1012      6.25          17        17    -   
           NZ_CAAGEU010000058.1_ORF1018   0.74           2         2    -   
           NZ_CAAGBZ010000129.1_ORF583     1.1           3         3    -   

                                        TaxID Internal Node?  
SampleName Scientific Name                                    
SRR5368774 NZ_CABJFM010000001.1_ORF1622   755          False  
           757                            757           True  
           NZ_VZBX01000029.1_ORF1012     1212          False  
           NZ_CAAGEU010000058.1_ORF1018  1215          False  
           NZ_CAAGBZ010000129.1_ORF583   1216          False

In [135]:
parser[parser["Clade Reads"]!=parser["ReadCount"]]

%Reads Clade Reads ReadCount Rank TaxID  \
SampleName Scientific Name                                           
SRR5368774 160               0.74           2         1    -   160   
           3198              1.47           4         2    -  3198   
SRR5368778 757               0.37           4         3    -   757   
           198              20.65         224        37    -   198   
           160               0.55           6         4    -   160   
...                           ...         ...       ...  ...   ...   
SRR5368905 198              75.22        1217       105    -   198   
           346               0.49           8         2    -   346   
           757               0.87          14         1    -   757   
SRR5368907 187              47.34          80         1    -   187   
           198               42.6          72         9    -   198   

                           Internal Node?  
SampleName Scientific Name                 
SRR5368774 160                       True  
           3198                      True  
SRR5368778 757                       True  
           198                       True  
           160                       True  
...                                   ...  
SRR5368905 198                       True  
           346                       True  
           757                       True  
SRR5368907 187                       True  
           198                       True  

[168 rows x 6 columns]

In [ ]:
# Calculate Effective Transcript Lengths

In [ ]:
parser.index